In [1]:
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt; 
import matplotlib.image as image
%matplotlib inline
from sklearn.metrics import confusion_matrix
import cPickle
import re,csv
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [2]:
import sys
sys.path.append('/Users/tgn/ConfigArgParse-0.10.0/build/lib')
sys.path.append('/Users/tgn/neon/build/lib')

In [3]:
from neon.backends import gen_backend
be = gen_backend(backend='cpu',batch_size=5)
print be

In [4]:
import logging
from neon.callbacks.callbacks import Callbacks
from neon.data import ArrayIterator
from neon.initializers import Gaussian
from neon.layers import GeneralizedCost, Affine
from neon.models import Model
from neon.optimizers import GradientDescentMomentum
from neon.transforms import Rectlin, Logistic, CrossEntropyBinary, Misclassification, MeanSquared, Accuracy,Softmax
from neon.util.argparser import NeonArgparser
from neon.util.compat import PY3, range

In [5]:
def get_idx_from_sent(sent, word_idx_map, max_l=51, k=300, filter_h=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    pad = filter_h - 1
    for i in xrange(pad):
        x.append(0)
    words = sent.split()
    for word in words:
        if word in word_idx_map and len(x)< max_l+2*pad:
            x.append(word_idx_map[word])
    while len(x) < max_l+2*pad:
        x.append(0)
    return x

In [6]:
def make_idx_data_cv(revs, word_idx_map,w, cv, max_l=51, k=300, filter_h=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    eleTy={}
    counter=0
    train, test = [], []
    train_l,test_l=[],[]
    x_train,x_test=[],[]
    for rev in revs:
        counter+=1
        sent = get_idx_from_sent(rev["text"], word_idx_map, max_l, k, filter_h)
        try:eleTy[len(sent)]+=1
        except KeyError: eleTy.update({(len(sent)):1})
        if rev["split"]==cv:
            test.append(w[sent])
            test_l.append(rev["y"])
        else:
            train.append(w[sent])
            train_l.append(rev["y"])
    #pp.pprint(train)
    '''
    print 'type', type(train)
    eleTy={}
    print 'element types'
    for x in train:matrix(y_true, y_pred[, ...])
        for y in x:
            try:eleTy[str(type(x))]+=1
            except KeyError: eleTy.update({str(type(x)):1})
    #pp.pprint(eleTy)
    try: train = np.asarray(train,dtype="int")
    except ValueError:
            train = np.asarray(train,dtype="object")
            print 'train shape', train.shape
    try: test = np.asarray(test,dtype="int")
    except ValueError:
        test = np.asarray(test,dtype="object")
        print 'test shape', test.shape
    '''
    pp.pprint(eleTy)
    print 'counter',counter
    print 'sent last'
    print sent[-1]
    train = np.asarray(train)
    test = np.asarray(test)
    test_l=np.asarray(test_l)
    train_l=np.asarray(train_l)
    for i in range(train.shape[0]):
        k=[]
        for j in range(300):
            k.append(np.mean(train[i][:,j]))
        x_train.append(k)
            
    for i in range(test.shape[0]):
        k=[]
        for j in range(300):
            k.append(np.mean(test[i][:,j]))
        x_test.append(k)
    x_train=np.asarray(x_train)
    x_test=np.asarray(x_test)
    
    return x_train,train_l,x_test,test_l

In [7]:
x = cPickle.load(open("mr.g","rb"))

In [8]:
revs, W, W2, word_idx_map, vocab, corpusMaxL, classlist= x[0], x[1], x[2], x[3], x[4],x[5],x[6]

In [11]:
x_train, y_train, x_test, y_test=make_idx_data_cv(revs, word_idx_map, W, 1, max_l=1500,k=300, filter_h=5)

{   1508: 500}
counter 500
sent last
0


In [12]:
print y_test

[5 6 6 3 3 6 5 6 3 6 3 6 6 6 5 6 6 0 6 6 6 3 6 2 6 6 6 1 6 6 0 6 6 5 6 6 5
 4 1 6 6 6 1 6 5 1 6 6 5 4 6 3 3]


In [13]:
from neon.data import ArrayIterator
train_set=ArrayIterator(x_train, y_train, nclass=12)
test_set=ArrayIterator(x_test, y_test, nclass=12)

In [15]:
result=[]
mean_error=[]
for j in range(3):
    init_norm = Gaussian(loc=0.0, scale=0.01)
    layers = [Affine(nout=150, init=init_norm, activation=Rectlin()),
              Affine(nout=12,init=init_norm,activation=Softmax())]
    DNN = Model(layers=layers)

    cost = GeneralizedCost(costfunc=MeanSquared())
    optimizer=GradientDescentMomentum(learning_rate=0.01,momentum_coef=0.9)
    callbacks=Callbacks(DNN, eval_set=test_set)
    DNN.fit(train_set, optimizer=optimizer, cost=cost, callbacks=callbacks, num_epochs=80)
    predic = DNN.get_outputs(test_set)
    error=DNN.eval(test_set, metric=Misclassification())*100
    print('Misclassification error = %.1f%%' % error)
    print predic
    result.append(error)
print result
print mean_error

Epoch 0   [Train |████████████████████|   90/90   batches, 0.04 cost, 0.18s]
Epoch 1   [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 2   [Train |████████████████████|   90/90   batches, 0.04 cost, 0.19s]
Epoch 3   [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 4   [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 5   [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 6   [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 7   [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 8   [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 9   [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 10  [Train |████████████████████|   89/89   batches, 0.04 cost, 0.20s]
Epoch 11  [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]
Epoch 12  [Train |████████████████████|   89/89   batches, 0.04 cost, 0.19s]